In [7]:
import pickle
import cv2
import numpy as np
import mediapipe as mp
from pathlib import Path
import pandas as pd
from PIL import Image, ImageDraw, ImageFont

In [12]:
# 학습 시 사용한 관절 인덱스 (MediaPipe Pose 인덱스)
JOINT_INDICES = {
    'shoulder_left': 11,
    'shoulder_right': 12,
    'elbow_left': 13,
    'elbow_right': 14,
    'wrist_left': 15,
    'wrist_right': 16,
    'hip_left': 23,
    'hip_right': 24,
    'knee_left': 25,
    'knee_right': 26,
    'ankle_left': 27,
    'ankle_right': 28,
    'heel_left': 29,
    'heel_right': 30,
    'toe_left': 31,
    'toe_right': 32
}

# 관절 이름 리스트 (순서 중요!)
JOINT_NAMES = [
    'shoulder_left', 'shoulder_right',
    'elbow_left', 'elbow_right',
    'wrist_left', 'wrist_right',
    'hip_left', 'hip_right',
    'knee_left', 'knee_right',
    'ankle_left', 'ankle_right',
    'heel_left', 'heel_right',
    'toe_left', 'toe_right'
]

def extract_landmarks_to_features(results):
    """
    MediaPipe 결과에서 학습 시 사용한 16개 관절의 x, y, z 좌표만 추출
    학습 시와 동일한 형태로 변환 (48개 특징: 16개 관절 × 3개 좌표)
    """
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        features = []
        
        # 학습 시 사용한 관절 순서대로 x, y, z 추출
        for joint_name in JOINT_NAMES:
            joint_idx = JOINT_INDICES[joint_name]
            lm = landmarks[joint_idx]
            features.extend([lm.x, lm.y, lm.z])
        
        return np.array(features, dtype=np.float32)
    else:
        return None

def create_feature_dataframe(features):
    """
    특징 벡터를 학습 시와 동일한 컬럼 이름을 가진 DataFrame으로 변환
    """
    # 컬럼 이름 생성: shoulder_left_x, shoulder_left_y, shoulder_left_z, ...
    columns = []
    for joint_name in JOINT_NAMES:
        columns.extend([f'{joint_name}_x', f'{joint_name}_y', f'{joint_name}_z'])
    
    # DataFrame 생성
    df = pd.DataFrame([features], columns=columns)
    return df

def put_korean_text(img, text, position, font_size=40, color=(255, 255, 255), bg_color=None):
    """
    OpenCV 이미지에 한글 텍스트를 표시하는 함수
    PIL을 사용하여 한글 폰트로 텍스트를 렌더링한 후 OpenCV 이미지에 합성
    """
    # OpenCV 이미지를 PIL 이미지로 변환 (BGR -> RGB)
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    
    # 한글 폰트 로드 시도
    try:
        # Windows 기본 한글 폰트 경로들
        font_paths = [
            "C:/Windows/Fonts/malgun.ttf",  # 맑은 고딕
            "C:/Windows/Fonts/gulim.ttc",   # 굴림
            "C:/Windows/Fonts/batang.ttc",  # 바탕
            "C:/Windows/Fonts/gungsuh.ttc",  # 궁서
        ]
        
        font = None
        for font_path in font_paths:
            try:
                font = ImageFont.truetype(font_path, font_size)
                break
            except:
                continue
        
        # 폰트를 찾지 못한 경우 기본 폰트 사용
        if font is None:
            font = ImageFont.load_default()
    except:
        font = ImageFont.load_default()
    
    # 텍스트 크기 계산
    bbox = draw.textbbox((0, 0), text, font=font)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]
    
    x, y = position
    
    # 배경 그리기
    if bg_color:
        padding = 10
        bg_rect = [
            x - padding,
            y - padding,
            x + text_width + padding,
            y + text_height + padding
        ]
        # PIL은 RGB 형식이므로 BGR을 RGB로 변환
        bg_rgb = (bg_color[2], bg_color[1], bg_color[0])
        draw.rectangle(bg_rect, fill=bg_rgb)
    
    # 텍스트 그리기 (PIL은 RGB 형식이므로 BGR을 RGB로 변환)
    text_rgb = (color[2], color[1], color[0])
    draw.text((x, y), text, font=font, fill=text_rgb)
    
    # PIL 이미지를 OpenCV 이미지로 변환 (RGB -> BGR)
    img_with_text = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
    
    return img_with_text

def process_video_with_model(video_path, model_path, scaler_path=None, target_width=None):
    """
    동영상 파일을 실시간으로 재생하면서 각 프레임에서 낙상/정상을 판단하고 결과를 화면에 표시
    
    Parameters:
    -----------
    video_path : str or Path
        입력 동영상 파일 경로
    model_path : str or Path
        저장된 모델 파일 경로 (.pkl)
    scaler_path : str or Path, optional
        저장된 스케일러 파일 경로 (.pkl) - 학습 시 스케일러를 사용했다면 필수
    target_width : int, optional
        표시할 동영상 너비 (None이면 원본 크기)
    """
    # 경로 변환
    video_path = Path(video_path)
    model_path = Path(model_path)
    
    if not video_path.exists():
        raise FileNotFoundError(f"동영상 파일을 찾을 수 없습니다: {video_path}")
    if not model_path.exists():
        raise FileNotFoundError(f"모델 파일을 찾을 수 없습니다: {model_path}")
    
    # 모델 로드
    print(f"모델 로딩 중: {model_path}")
    with open(model_path, 'rb') as f:
        svc_model = pickle.load(f)
    print("모델 로드 완료!")
    
    # 스케일러 로드 (있는 경우)
    scaler = None
    if scaler_path:
        scaler_path = Path(scaler_path)
        if scaler_path.exists():
            print(f"스케일러 로딩 중: {scaler_path}")
            with open(scaler_path, 'rb') as f:
                scaler = pickle.load(f)
            print("스케일러 로드 완료!")
        else:
            print(f"경고: 스케일러 파일을 찾을 수 없습니다: {scaler_path}")
    
    # 동영상 열기
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise SystemError(f"동영상을 열 수 없습니다: {video_path}")
    
    # 동영상 정보 가져오기
    fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"동영상 정보: {width}x{height}, {fps:.2f} FPS, 총 {total_frames} 프레임")
    print("재생 중... (q 키를 누르면 종료)")
    
    # 표시할 크기 결정
    if target_width and width > target_width:
        scale = target_width / width
        display_size = (int(width * scale), int(height * scale))
    else:
        display_size = (width, height)
    
    # MediaPipe Pose 초기화
    with mp_pose.Pose(
        static_image_mode=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5,
        model_complexity=1
    ) as pose:
        
        frame_count = 0
        
        while True:
            ret, frame = cap.read()
            if not ret:
                print("동영상 재생 완료!")
                break
            
            frame_count += 1
            
            # 표시할 프레임 크기 조정
            if display_size != (width, height):
                display_frame = cv2.resize(frame, display_size)
            else:
                display_frame = frame.copy()
            
            # RGB로 변환 (MediaPipe는 RGB 사용)
            rgb_frame = cv2.cvtColor(display_frame, cv2.COLOR_BGR2RGB)
            
            # 관절 좌표 추출
            results = pose.process(rgb_frame)
            
            # 스켈레톤 그리기
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    display_frame,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
                )
                
                # 관절 좌표를 특징 벡터로 변환 (16개 관절 × 3개 좌표 = 48개)
                features = extract_landmarks_to_features(results)
                
                if features is not None:
                    try:
                        # DataFrame 생성 (학습 시와 동일한 컬럼 이름)
                        features_df = create_feature_dataframe(features)
                        
                        # 스케일링 (스케일러가 있는 경우)
                        if scaler is not None:
                            features_scaled = scaler.transform(features_df)
                            features_df = pd.DataFrame(features_scaled, columns=features_df.columns)
                        
                        # 모델 예측
                        prediction = svc_model.predict(features_df)[0]
                        proba = None
                        
                        # 확률 예측 시도 (probability=True인 경우)
                        try:
                            proba = svc_model.predict_proba(features_df)[0]
                        except AttributeError:
                            pass
                        
                        # 예측 결과에 따라 텍스트와 색상 설정
                        if prediction == 0:  # 정상
                            text = "정상"
                            text_color = (0, 255, 0)  # 초록색 (BGR)
                        else:  # 낙상
                            text = "낙상"
                            text_color = (0, 0, 255)  # 빨간색 (BGR)
                        
                        # 확률 정보 추가 (있는 경우)
                        if proba is not None:
                            confidence = proba[prediction] * 100
                            text_with_proba = f"{text} ({confidence:.1f}%)"
                        else:
                            text_with_proba = text
                        
                        # 한글 텍스트 표시
                        display_frame = put_korean_text(
                            display_frame,
                            text_with_proba,
                            position=(20, 20),
                            font_size=50,
                            color=text_color,
                            bg_color=(0, 0, 0)  # 검은 배경
                        )
                        
                    except Exception as e:
                        print(f"프레임 {frame_count} 예측 오류: {e}")
                        import traceback
                        traceback.print_exc()
                        display_frame = put_korean_text(
                            display_frame,
                            "예측 실패",
                            position=(20, 20),
                            font_size=40,
                            color=(0, 0, 255),
                            bg_color=(0, 0, 0)
                        )
            else:
                # 관절이 감지되지 않은 경우
                display_frame = put_korean_text(
                    display_frame,
                    "관절 미감지",
                    position=(20, 20),
                    font_size=40,
                    color=(128, 128, 128),
                    bg_color=(0, 0, 0)
                )
            
            # 프레임 번호 표시
            progress_text = f"Frame: {frame_count}/{total_frames}"
            cv2.putText(
                display_frame,
                progress_text,
                (20, display_frame.shape[0] - 20),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (255, 255, 255),
                2
            )
            
            # 화면에 표시
            cv2.imshow('Fall Detection - 실시간 감지', display_frame)
            
            # FPS에 맞춰 대기 (q 키로 종료)
            delay = int(1000 / fps)  # 밀리초
            if cv2.waitKey(delay) & 0xFF == ord('q'):
                print("사용자에 의해 중단되었습니다.")
                break
    
    # 정리
    cap.release()
    cv2.destroyAllWindows()
    print("프로그램 종료")

In [23]:
#  MediaPipe 모델 초기화
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

video_path = r'C:\Users\ASUS\Desktop\딥러닝 프로젝트\00135_H_A_SY_C4.mp4'
# 동영상 실행하기
process_video_with_model(
    video_path=video_path,
    model_path="svc_model.pkl"
)

모델 로딩 중: svc_model.pkl
모델 로드 완료!
동영상 정보: 1280x720, 10.00 FPS, 총 100 프레임
재생 중... (q 키를 누르면 종료)
동영상 재생 완료!
프로그램 종료
